# Advent of Haystack: Day 9

In this challenge, you'll help Santa build a self-reflecting gift selection agent using Haystack and MongoDB Atlas! 🎅

The agent will help optimize gift selections based on children's wishlists and budget constraints, using MongoDB Atlas vector search for semantic matching and implementing self-reflection to ensure the best possible gift combinations.

**Components to use in this challenge:**
- [`OpenAITextEmbedder`](https://docs.haystack.deepset.ai/docs/openaitextembedder) for  query embedding
- [`MongoDBAtlasEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/) for finding relevant gifts
- [`PromptBuilder`](https://docs.haystack.deepset.ai/docs/promptbuilder) for creating the prompt
- [`OpenAIGenerator`](https://docs.haystack.deepset.ai/docs/openaigenerator) for  generating responses
- Custom `GiftChecker` component for self-reflection

## Setup

First, let's install the required packages:

In [ ]:
!pip install haystack-ai mongodb-atlas-haystack tiktoken datasets colorama -q --upgrade openai

## Configure Environment

- [OpenAI API Key](https://platform.openai.com/api-keys) if you'd like to use OpenAI embedding and text generation models
- [MongoDB Atlas project](https://www.mongodb.com/docs/atlas/getting-started/) with an Atlas cluster (free tier works). [Detailed Tutorial](https://www.mongodb.com/docs/guides/atlas/cluster/#create-a-cluster)
- Get your [connection string](https://www.mongodb.com/docs/atlas/tutorial/connect-to-your-cluster/#connect-to-your-atlas-cluster) and have `0.0.0.0/0` address in your network access list.
- Connection string looks like this `mongodb+srv://<db_username>:<db_password>@<clustername>.xxxxx.mongodb.net/?retryWrites=true...`

Set up your MongoDB Atlas and OpenAI credentials:

In [8]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [9]:
import os

os.environ['MONGO_CONNECTION_STRING']=os.getenv("MONGO_CONNECTION_STRING")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

## Sample Gift Dataset

Here's a starter dataset of gifts. Feel free to expand it with more options!

In [10]:
dataset = {
    "train": [
    {
        "title": "LEGO Star Wars Set",
        "price": "$49.99",
        "description": "Build your own galaxy with this exciting LEGO Star Wars set",
        "category": "Toys",
        "age_range": "7-12"
    },
    {
        "title": "Remote Control Car",
        "price": "$29.99",
        "description": "Fast and fun RC car with full directional control",
        "category": "Toys",
        "age_range": "6-10"
    },
    {
        "title": "Art Set",
        "price": "$24.99",
        "description": "Complete art set with paints, brushes, and canvas",
        "category": "Arts & Crafts",
        "age_range": "5-15"
    },
    {
        "title": "Science Kit",
        "price": "$34.99",
        "description": "Educational science experiments kit",
        "category": "Educational",
        "age_range": "8-14"
    },
    {
        "title": "Dollhouse",
        "price": "$89.99",
        "description": "Beautiful wooden dollhouse with furniture",
        "category": "Toys",
        "age_range": "4-10"
    },
    {
        "title": "Building Blocks Set",
        "price": "$39.99",
        "description": "Classic wooden building blocks in various shapes and colors",
        "category": "Educational",
        "age_range": "3-8"
    },
    {
        "title": "Board Game Collection",
        "price": "$44.99",
        "description": "Set of 5 classic family board games",
        "category": "Games",
        "age_range": "6-99"
    },
    {
        "title": "Puppet Theater",
        "price": "$59.99",
        "description": "Wooden puppet theater with 6 hand puppets",
        "category": "Creative Play",
        "age_range": "4-12"
    },
    {
        "title": "Robot Building Kit",
        "price": "$69.99",
        "description": "Build and program your own robot with this STEM kit",
        "category": "Educational",
        "age_range": "10-16"
    },
    {
        "title": "Play Kitchen",
        "price": "$79.99",
        "description": "Realistic play kitchen with sounds and accessories",
        "category": "Pretend Play",
        "age_range": "3-8"
    },
    {
        "title": "Musical Instrument Set",
        "price": "$45.99",
        "description": "Collection of kid-friendly musical instruments",
        "category": "Music",
        "age_range": "3-10"
    },
    {
        "title": "Sports Equipment Pack",
        "price": "$54.99",
        "description": "Complete set of kids' sports gear including ball, bat, and net",
        "category": "Sports",
        "age_range": "6-12"
    },
    {
        "title": "Magic Tricks Kit",
        "price": "$29.99",
        "description": "Professional magic set with instruction manual",
        "category": "Entertainment",
        "age_range": "8-15"
    },
    {
        "title": "Dinosaur Collection",
        "price": "$39.99",
        "description": "Set of 12 detailed dinosaur figures with fact cards",
        "category": "Educational",
        "age_range": "4-12"
    },
    {
        "title": "Craft Supply Bundle",
        "price": "$49.99",
        "description": "Comprehensive craft supplies including beads, yarn, and tools",
        "category": "Arts & Crafts",
        "age_range": "6-16"
    },
    {
        "title": "Coding for Kids Set",
        "price": "$64.99",
        "description": "Interactive coding kit with programmable robot and game cards",
        "category": "STEM",
        "age_range": "8-14"
    },
    {
        "title": "Dress Up Trunk",
        "price": "$49.99",
        "description": "Collection of costumes and accessories for imaginative play",
        "category": "Pretend Play",
        "age_range": "3-10"
    },
    {
        "title": "Microscope Kit",
        "price": "$59.99",
        "description": "Real working microscope with prepared slides and tools",
        "category": "Science",
        "age_range": "10-15"
    },
    {
        "title": "Outdoor Explorer Kit",
        "price": "$34.99",
        "description": "Nature exploration set with binoculars, compass, and field guide",
        "category": "Outdoor",
        "age_range": "7-12"
    },
    {
        "title": "Art Pottery Studio",
        "price": "$69.99",
        "description": "Complete pottery wheel set with clay and glazing materials",
        "category": "Arts & Crafts",
        "age_range": "8-16"
    }
]
}

## Initialize MongoDB Atlas

First, we need to set up our MongoDB Atlas collection and create a vector search index. This step is crucial for enabling semantic search capabilities:

In [6]:
 # Create collection gifts and add the vector index

from pymongo import MongoClient
from bson import json_util
from pymongo.operations import SearchIndexModel
import json
import time

client = MongoClient(os.environ['MONGO_CONNECTION_STRING'])
db = client['santa_workshop']
collection = db['gifts']

db.create_collection("gifts")


## create index
search_index_model = SearchIndexModel(
  definition={
    "fields": [
      {
        "type": "vector",
        "numDimensions": 1536,
        "path": "embedding",
        "similarity": "cosine"
      },
    ]
  },
  name="vector_index",
  type="vectorSearch",
)
result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True
while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")
client.close()

New search index named vector_index is building.
Polling to check if the index is ready. This may take up to a minute.
vector_index is ready for querying.


## Initialize Document Store and Index Documents

Now let's set up the [MongoDBAtlasDocumentStore](https://docs.haystack.deepset.ai/docs/mongodbatlasdocumentstore) and index our gift data:

In [7]:
from haystack import Pipeline, Document
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from bson import json_util

# Initialize document store
document_store = MongoDBAtlasDocumentStore(
    database_name="santa_workshop",
    collection_name="gifts",
    vector_search_index="vector_index"
)

# Convert dataset to Haystack documents
insert_data = [
    Document(content=gift['title'], meta=json_util.loads(json_util.dumps(gift)))
    for gift in dataset['train']
]

# Create and run indexing pipeline
doc_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)
doc_embedder = OpenAIDocumentEmbedder(
    model="text-embedding-3-small",
    meta_fields_to_embed=["description"]  # Include description in embeddings
)

indexing_pipe = Pipeline()
indexing_pipe.add_component(instance=doc_embedder, name="doc_embedder")
indexing_pipe.add_component(instance=doc_writer, name="doc_writer")
indexing_pipe.connect("doc_embedder.documents", "doc_writer.documents")

indexing_pipe.run({"doc_embedder": {"documents": insert_data}})

/opt/homebrew/Caskroom/miniforge/base/envs/advent_haystack_24/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Calculating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


{'doc_embedder': {'meta': {'model': 'text-embedding-3-small',
   'usage': {'prompt_tokens': 241, 'total_tokens': 241}}},
 'doc_writer': {'documents_written': 20}}

## Create Gift Selection Pipeline

Now comes the fun part! Create a pipeline that can:
1. Take a gift request query
2. Find relevant gifts using vector search
3. Self-reflect on selections to optimize for budget and preferences

Here's the basic structure to get you started:

In [56]:
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.embedders import OpenAITextEmbedder
from colorama import Fore
from typing import List
from haystack import component

@component
class GiftChecker:
    @component.output_types(gifts_to_check=str, gifts=str)
    def run(self, replies: List[str]):
        print("Previous replies: ",replies[0])
        print("Is DONE in replies[0]: ", "DONE" in replies[0])
        if "DONE" in replies[0]:
            return {"gifts": replies[0].replace("DONE", "")}
        else:
            print(Fore.RED + "Reflecting on gift selection\n", replies[0])
            return {"gifts_to_check": replies[0]}

        

prompt_template = """
    You are Santa's gift selection assistant. Below you have a list of available gifts with their prices.
    Based on the child's wishlist and budget, suggest appropriate gifts that maximize joy while staying within budget.

    Available Gifts:
    {% for doc in documents %}
        Gift: {{ doc.content }}
        Price: {{ doc.meta['price']}}
        Age Range: {{ doc.meta['age_range']}}
    {% endfor %}

    Query: {{query}}
    {% if gifts_to_check %}
        Previous gift selection: {{gifts_to_check[0]}}
        Can we optimize this selection for better value within budget?
        Things to check for:
        - Total price should be within budget
        - Age range should be appropriate
        - Category should match the child's interests
        If there are still multiple valid options, choose the one that maximizes joy
        If that choice is optimal, you MUST include the word 'DONE' in your reply and return the selection
        If not, suggest a better combination
    {% endif %}

    Gift Selection:
"""

gift_pipeline = Pipeline(max_runs_per_component=5)

query_embedder = OpenAITextEmbedder(
    model="text-embedding-3-small"
)
embed_retriever = MongoDBAtlasEmbeddingRetriever(
    document_store=document_store,
)

prompt_builder = PromptBuilder(template=prompt_template)
gift_checker = GiftChecker()
generator = OpenAIGenerator()

gift_pipeline.add_component(instance=query_embedder, name="query_embedder")
gift_pipeline.add_component(instance=embed_retriever, name="embed_retriever")
gift_pipeline.add_component(instance=prompt_builder, name="prompt_builder")
gift_pipeline.add_component(instance=gift_checker, name="gift_checker")
gift_pipeline.add_component(instance=generator, name="generator")

gift_pipeline.connect("query_embedder.embedding", "embed_retriever.query_embedding")
gift_pipeline.connect("embed_retriever", "prompt_builder")
gift_pipeline.connect("prompt_builder.prompt", "generator.prompt")
gift_pipeline.connect("generator.replies", "gift_checker.replies")
gift_pipeline.connect("gift_checker.gifts_to_check", "prompt_builder.gifts_to_check")


🚅 Components
  - query_embedder: OpenAITextEmbedder
  - embed_retriever: MongoDBAtlasEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - gift_checker: GiftChecker
  - generator: OpenAIGenerator
🛤️ Connections
  - query_embedder.embedding -> embed_retriever.query_embedding (List[float])
  - embed_retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)
  - gift_checker.gifts_to_check -> prompt_builder.gifts_to_check (str)
  - generator.replies -> gift_checker.replies (List[str])

## Test Your Agent

Once you've built your pipeline, test it with different scenarios:

In [58]:
# Test queries to try:
queries = [
    "Find gifts for a 9-year-old who loves science and building things. Budget: $100",
    "Need gifts for a creative 6-year-old interested in art. Budget: $50",
    "Looking for educational toys for a 12-year-old. Budget: $75"
]

query = queries[0] 

# Run the pipeline and print the results
result = gift_pipeline.run(data={"query_embedder": {"text": query}, "prompt_builder": {"query": query}})
print(Fore.GREEN + result['gift_checker']['gifts'])


Previous replies:  For a 9-year-old who loves science and building things with a budget of $100, here are the best gift options from the available list:

1. **Science Kit**
   - Price: $34.99
   - Age Range: 8-14

2. **Outdoor Explorer Kit**
   - Price: $34.99
   - Age Range: 7-12

3. **Building Blocks Set**
   - Price: $39.99
   - Age Range: 3-8

### Gift Combinations:

- **Option 1:**
  - Science Kit ($34.99) + Outdoor Explorer Kit ($34.99) = **Total: $69.98**
- **Option 2:**
  - Science Kit ($34.99) + Building Blocks Set ($39.99) = **Total: $74.98**
- **Option 3:**
  - Outdoor Explorer Kit ($34.99) + Building Blocks Set ($39.99) = **Total: $74.98**

These options would maximize joy by providing a mix of science exploration and building activities while staying well within the budget. 

### Recommended Gift Selection:
- **Science Kit** ($34.99)
- **Outdoor Explorer Kit** ($34.99) 

Total: **$69.98**

This combination allows for both scientific exploration and outdoor play, appealing 

## Next Steps (Bonus)

Once you have the basic pipeline working, try these extensions:
1. Add sentiment analysis to evaluate gift joy potential
2. Implement gift bundle optimization
3. Add previous gift tracking to ensure variety
4. Enhance the prompt to consider seasonal availability
5. Add gift wrapping suggestions based on the selected gifts

Good luck! 🎄✨